## Train a model 

In [1]:
%%bash
mlflow run bike-rental-regression -P train_year=2011 -P train_month=1 -P test_year=2011 -P test_month=3 -P dataset_path=train_bike.csv

2021/11/06 23:14:34 INFO mlflow.projects.utils: === Created directory /tmp/tmpj9frplnw for downloading remote URIs passed to arguments of type 'path' ===
2021/11/06 23:14:34 INFO mlflow.projects.backend.local: === Running command 'source /home/andrea/miniconda3/bin/../etc/profile.d/conda.sh && conda activate mlflow-3ba878fc7c575322867d4954b82239190f6a5ce6 1>&2 && python train.py 2011 1 2011 3 /home/andrea/demo-ATSS/train_bike.csv' in run with ID '5ff1f681065a4481915308366259ade3' === 
/home/andrea/miniconda3/envs/mlflow-3ba878fc7c575322867d4954b82239190f6a5ce6/lib/python3.9/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) t

## Package the model in a Docker image and register it in the local registry

In [6]:
%%bash
mlflow models build-docker --model-uri runs:/5ff1f681065a4481915308366259ade3/model --name "my-image-name"

/tmp/tmpq2bv095r/
/tmp/tmpq2bv095r/model_dir
/tmp/tmpq2bv095r/model_dir/model
/tmp/tmpq2bv095r/model_dir/model/model.pkl
/tmp/tmpq2bv095r/model_dir/model/conda.yaml
/tmp/tmpq2bv095r/model_dir/model/MLmodel
/tmp/tmpq2bv095r/model_dir/model/requirements.txt
/tmp/tmpq2bv095r/model_dir/model/input_example.json
/tmp/tmpq2bv095r/Dockerfile


2021/11/06 23:18:00 INFO mlflow.models.cli: Selected backend for flavor 'python_function'
2021/11/06 23:18:00 INFO mlflow.models.docker_utils: Building docker image with name my-image-name
#1 [internal] load build definition from Dockerfile
#1 sha256:aba316cc577c7efa97be12545bce22f670c920d3f2a6d6fd304dfcaf9c3a1ac5
#1 DONE 0.0s

#1 [internal] load build definition from Dockerfile
#1 sha256:aba316cc577c7efa97be12545bce22f670c920d3f2a6d6fd304dfcaf9c3a1ac5
#1 transferring dockerfile: 1.58kB done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:f3c382f0b0b2ead385efc877ee5f39516b09626abc763adee93a75154c0ba4fb
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:18.04
#3 sha256:ae46bbb1b755529d0da663ca0256a22acd7c9fe21844946c149800baa67c4e4b
#3 ...

#4 [auth] library/ubuntu:pull token for registry-1.docker.io
#4 sha256:17371cd6af1d9c12f342e8e7032fad61af264b8d1e977e88472b8d343a7983a2
#4 DONE 0.0s

#3 [internal] load metadata for docker

## Start the docker container

Run this command in a separate shell, because it needs to run in the background. It will be killed with ^C.

```
docker run -p 5001:8080 "my-image-name"
```

## Try to get a prediction

In [9]:
import requests
import json
import pandas as pd

In [24]:
dataset = pd.read_csv('train_bike.csv', header=0,  sep=',',
                      parse_dates=['datetime'], index_col='datetime').drop(
    columns=['casual', 'registered']
)
test_year = 2011
test_month = 3
test: pd.DataFrame = dataset.loc[
    lambda df: (df.index.year == test_year) & (df.index.month == test_month)
]
def time_features(input_df):
    return input_df.assign(
        month = lambda df: df.index.month,
        day = lambda df: df.index.day,
        hour = lambda df: df.index.hour,
    )
test.pipe(time_features).drop(columns='count').head().to_json(orient='split', index=False)

predictions = requests.post(
    'http://127.0.0.1:5001/invocations',
    json=json.loads(
        test.pipe(time_features)
        .drop(columns='count')
        .to_json(orient='split', index=False)
    )
).json()

from sklearn.metrics import r2_score

r2_score(test.pipe(time_features)['count'].values, predictions)

0.6026563896049335